In [55]:
# import necessary modules
import pandas as pd
import requests
import zipfile
import os

In [56]:
# Set display option to show all columns
pd.set_option('display.max_columns', None)

In [32]:
# create a folder to hold the data
data_folder = 'Data'
# Check if the folder exists
if not os.path.exists(data_folder):
    # Create the folder
    os.makedirs(data_folder)
    print(f'{data_folder} folder created successfully')
else:
    print(f'{data_folder} folder already exists')

Data folder created successfully


In [12]:
# url of the API
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# retrieve the metadata for pcard-expenditures package and its resources
url = base_url + "/api/3/action/package_show"
params = { "id": "pcard-expenditures"}
package = requests.get(url, params = params, verify=False).json()

C:\Users\mmangoli\AppData\Local\Programs\Python\Python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ckan0.cf.opendata.inter.prod-toronto.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [13]:
package

{'help': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/help_show?name=package_show',
 'success': True,
 'result': {'author': 'pcard@toronto.ca',
  'author_email': 'pcard@toronto.ca',
  'civic_issues': 'Fiscal responsibility',
  'creator_user_id': '329e1506-b545-4fc7-a4ea-e614f220eea7',
  'dataset_category': 'Document',
  'date_published': '2019-07-23 17:52:47.150105',
  'excerpt': 'The dataset contains details of all purchases made by City staff members using City-issued credit cards also referred to as PCards or purchasing cards.   ',
  'formats': 'ZIP,XLS',
  'id': 'ebc3f9c2-2f80-4405-bf4f-5fb309581485',
  'information_url': 'http://www.toronto.ca/finance/index.htm',
  'is_retired': 'false',
  'isopen': False,
  'last_refreshed': '2019-09-03 19:44:00',
  'license_id': 'open-government-licence-toronto',
  'license_title': 'open-government-licence-toronto',
  'maintainer': None,
  'maintainer_email': 'pcard@toronto.ca',
  'metadata_created': '2022-03-10T19:34:25.429521'

In [14]:
package.keys()

dict_keys(['help', 'success', 'result'])

In [18]:
# get the urls with data
data_url = ''
for r in package['result']['resources']:
    print(r['format'],r['url'])
    if r['format'] == 'ZIP':
        data_url = r['url']

print(f'Link for data {data_url}')

XLS https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/ebc3f9c2-2f80-4405-bf4f-5fb309581485/resource/070bdbd3-9bae-4269-b096-e3a8bd7460c8/download/pcard_expenditures_readme.xls
ZIP https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/ebc3f9c2-2f80-4405-bf4f-5fb309581485/resource/d83a5249-fb07-4c38-9145-9e12a32ce1d4/download/expenditures.zip
Link for data https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/ebc3f9c2-2f80-4405-bf4f-5fb309581485/resource/d83a5249-fb07-4c38-9145-9e12a32ce1d4/download/expenditures.zip


In [33]:
# Send a GET request to the URL
data_response = requests.get(data_url, verify=False)

C:\Users\mmangoli\AppData\Local\Programs\Python\Python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ckan0.cf.opendata.inter.prod-toronto.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [43]:
# Save the content of the response to a file
downloaded_zip = 'expenditures.zip'
with open(downloaded_zip, "wb") as file:
    file.write(data_response.content)

In [44]:
# we want to extract the contents of the downloaded zip folder
# Open the ZIP file
with zipfile.ZipFile(downloaded_zip,'r') as zip_obj:
    # extract the data into the data folder we created
    zip_obj.extractall(data_folder)

In [57]:
# import data from the extracted files above

# an empty list that will hold our data
li_df = []
# list of files we extracted
li_files = os.listdir(data_folder)
print(f'we shall import data from {len(li_files)} files')

# initialize a counter
c = 0

for file in li_files:
    # import the data from the files
    df = pd.read_excel(data_folder + '\\' + file)
    # add a column to idenfity which file we extracted the data from
    df['Source_File_Name'] = file
    # add the data to our list of dataframes
    li_df.append(df)
    print(f'{c+1}. data from {file} imported successfully. Rows: {df.shape[0]}. Columns: {df.shape[1]}')
    c += 1

we shall import data from 103 files
1. data from PCard Expenses_201706.xlsx imported successfully. Rows: 6281. Columns: 17
2. data from PCard Expenses_201707.xlsx imported successfully. Rows: 6610. Columns: 17
3. data from PCard Expenses_201708.xlsx imported successfully. Rows: 5648. Columns: 17
4. data from PCard Expenses_201709.xlsx imported successfully. Rows: 4831. Columns: 17
5. data from PCard Expenses_201710.xlsx imported successfully. Rows: 5756. Columns: 17


KeyboardInterrupt: 

In [58]:
# combine the imported data frames into one
combined_df = pd.concat(li_df)
print(combined_df.shape)
combined_df.head()

(29126, 21)


,Division,Batch-Transaction ID,Transaction Date,Card Posting Dt,Merchant Name,Transaction Amt.,Trx Currency,Original Amount,Original Currency,G/L Account,G/L Account Description,Cost Centre /WBS Element / Order,Cost Centre /WBS Element / Order Description,Merchant Type,Merchant Type Description,Purpose,Source_File_Name,Batch Transaction ID,Cost Center / WBS Element / Order,Cost Center / WBS Element / Order Description,Exp Type Desc
0,PUBLIC HEALTH,4608-1,2017-06-15,2017-06-16,PAYPAL *OBC2012,50.00,CAD,50.00,CAD,4760,MEMBERSHIP FEES,PH3071,MATERNAL INFANT HEALTH PROGRAM SUPPORT,8641.0,"Associations - Civic, Social, and Frater",MEMBERSHIP FEE,PCard Expenses_201706.xlsx,NaN,NaN,NaN,NaN
1,PUBLIC HEALTH,4617-1,2017-06-21,2017-06-23,CHNC,423.75,CAD,423.75,CAD,4256,CONFERENCES/SEMINARS - REGISTRATION FEES,PH3071,MATERNAL INFANT HEALTH PROGRAM SUPPORT,8641.0,"Associations - Civic, Social, and Frater",CHNC CONFERENCE,PCard Expenses_201706.xlsx,NaN,NaN,NaN,NaN
2,PUBLIC HEALTH,4621-1,2017-06-26,2017-06-27,POST MD-CPD-UOFT,2601.00,CAD,2601.00,CAD,4256,CONFERENCES/SEMINARS - REGISTRATION FEES,PH3071,MATERNAL INFANT HEALTH PROGRAM SUPPORT,8220.0,"Colleges, Universities, Professional Sch",UOFT CONFERENCE,PCard Expenses_201706.xlsx,NaN,NaN,NaN,NaN
3,PUBLIC HEALTH,4626-1,2017-06-28,2017-06-29,EVENTBRITE/PERINATALMO,480.66,CAD,480.66,CAD,4256,CONFERENCES/SEMINARS - REGISTRATION FEES,PH3071,MATERNAL INFANT HEALTH PROGRAM SUPPORT,7922.0,Theatrical Producers (except Motion Pict,REGISTRATION FEES,PCard Expenses_201706.xlsx,NaN,NaN,NaN,NaN
4,NaN,NaN,NaT,NaT,NaN,3555.41,CAD,3555.41,CAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PCard Expenses_201706.xlsx,NaN,NaN,NaN,NaN
